In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r'E-691 Rank Student Scores.xlsx', usecols = 'A:B', header = 1, nrows = 13)
df.head()

,Student,Score
0,S1,23
1,S2,19
2,S3,15
3,S4,12
4,S5,12


In [3]:
ranks = (
    df
    .loc[:,['Score']]
    .drop_duplicates()
    .assign(Rank = lambda x: x.rank(ascending = True, method = 'min').astype(int))
)
ranks

,Score,Rank
0,23,5
1,19,4
2,15,3
3,12,2
5,8,1


In [4]:
result = (
    df.
    merge(ranks, on = 'Score', how = 'left')
    .groupby('Rank')
    .agg(
        Student = ('Student', lambda x: list(x)),
        Score = ('Score', lambda x: list(x)),
        Rank = ('Rank', lambda x: list(x) ),
        Rank2 = ('Rank', lambda x: list(range(1, len(x) + 1)))
    )
    .reset_index(drop = True)
    .explode(['Student', 'Score', 'Rank', 'Rank2'])
    .assign(
        Rank2 = lambda x: x['Rank'] + x['Rank2'] / 100,
        Index = lambda x: x['Student'].str.extract(r'(\d+)').astype(int)
    )
    .sort_values('Index')
    .loc[:,['Student', 'Score', 'Rank', 'Rank2']]
    .reset_index(drop = True)
)
result.head()

,Student,Score,Rank,Rank2
0,S1,23,5,5.01
1,S2,19,4,4.01
2,S3,15,3,3.01
3,S4,12,2,2.01
4,S5,12,2,2.02


Solution 2:

In [5]:
solution2 = df.copy()
solution2.head()

,Student,Score
0,S1,23
1,S2,19
2,S3,15
3,S4,12
4,S5,12


In [6]:
solution2 = (
    solution2
    .assign(Rank = lambda x: x['Score'].rank(ascending=False, method='dense').astype(int))
    .assign(Rank2 = lambda x: x['Rank'] + x.groupby('Score').cumcount().add(1) / 100)
)
solution2.head()

,Student,Score,Rank,Rank2
0,S1,23,1,1.01
1,S2,19,2,2.01
2,S3,15,3,3.01
3,S4,12,4,4.01
4,S5,12,4,4.02
